# Node Overlap

I gathered partially organized publicly available graphics. Around 60% of these graphics were created or opened on inkscape. It is likely that the groups within these graphics were created to make editing easier.

In [ ]:
# Loading dataset
from vectorrvnn.utils import *
from vectorrvnn.data import *
from vectorrvnn.utils import *
from vectorrvnn.baselines import *
from vectorrvnn.trainutils import *
from vectorrvnn.interfaces import *
from more_itertools import *
import svgpathtools as svg
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

DATA_DIR = '../data/PublicDomainVectors'

svgFiles = [f for f in allfiles(DATA_DIR) if f.endswith('svg')][:100]
dataset  = [SVGData(_) for _ in svgFiles]
# Filter out graphics with too many paths. 
dataset = [_ for _ in dataset if _.nPaths < 40] 

In [ ]:
# Visualize some groups
print(len(dataset))
sample = dataset[20:]
for _ in sample :
    plt.imshow(rasterize(_.doc, 200, 200))
    plt.show()
    print(_.svgFile)
    groupNodes = nonLeaves(_)
    groupNodes.remove(findRoot(_))
    pathSets = [_.nodes[n]['pathSet'] for n in groupNodes]
    nPs = len(pathSets)
    fig, axes = plt.subplots(1, nPs)
    psSample = random.sample(pathSets, k=nPs)
    print(nPs)
    for ps, ax in zip(psSample, axes) : 
        ax.imshow(rasterize(subsetSvg(_.doc, ps), 200, 200))
    plt.show()


## What are we measuring?

I compare three methods for graphic organization using this dataset. 

1. Ours
2. Fisher et. al.
3. Suggero

For each graphic in the dataset, I use one of the three methods to obtain a complete hierarchical organization, $T$. Then, for each group $G$ in the graphic, I find the node in the hierarchy that maximally overlaps with it. This score is used to rank the three methods. 

$$score(G, T) = max_{n \in V(T)} IoU(leaves(T, n), G)$$

In [ ]:
# Load our model
opts = Options().parse(testing=[
    '--checkpoints_dir', '../results',
    '--dataroot', '../data/All',
    '--embedding_size', '512',
    '--hidden_size', '1024',
    '--encoder_layers', '2',
    '--heads', '2',
    '--load_ckpt', 'obb-6/best_0-756-12-23-2021-18-13-36.pth',
    '--modelcls', 'OBBNet',
    '--name', 'test',
    '--phase', 'test',
])

model = buildModel(opts)

In [ ]:
from collections import defaultdict

def iou (a, b) : 
    return len(set(a).intersection(set(b))) / len(set(a).union(set(b)))

def logResult (T, method, methodName, results) : 
    T_ = method(T)
    gNodes = nonLeaves(T)
    gNodes.remove(findRoot(T))
    lens, scores = [], []
    for n in gNodes : 
        ps = T.nodes[n]['pathSet']
        maxIoU = max([iou(ps, T_.nodes[_]['pathSet']) for _ in T_.nodes])
        paths = cachedPaths(T.doc)
        lens.append(len(ps))
        scores.append(maxIoU)
    results[T][methodName] = dict(lens=lens, scores=scores, tree=T_)

In [ ]:
res1 = defaultdict(dict)
res2 = defaultdict(dict)

for k in tqdm(dataset) : 
    logResult(k, model.greedyTree, 'Ours', res1)
#     logResult(k, autogroup, 'Fisher et. al.', res1)
#     logResult(k, suggero, 'Suggero', res1)
    
# for k in tqdm(dataset[200:400]) : 
#     logResult(k, model.containmentGuidedTree, 'Ours', res2)
# #     logResult(k, autogroup, 'Fisher et. al.', res2)
# #     logResult(k, suggero, 'Suggero', res2)

## Average MaxIoU

Over 1000 groups are evaluated in this section

In [ ]:
ourScores, suggeroScores, fisherScores = [], [], []
ourLens, suggeroLens, fisherLens   = [], [], []

for k in res1.keys() : 
    if 'Ours' in res1[k] : 
        ourScores.extend(res1[k]['Ours']['scores'])
        ourLens.extend(res1[k]['Ours']['lens'])
#     if 'Fisher et. al.' in res1[k] : 
#         fisherScores.extend(res1[k]['Fisher et. al.']['scores'])
#         fisherLens.extend(res1[k]['Fisher et. al.']['lens'])
#     if 'Suggero' in res1[k] : 
#         suggeroScores.extend(res1[k]['Suggero']['scores'])
#         suggeroLens.extend(res1[k]['Suggero']['lens'])

print('Our\t average MaxIoU: ', '{:.3}'.format(np.mean(ourScores)))
# print('Fisher\t average MaxIoU: ', '{:.3}'.format(np.mean(fisherScores)))
# print('Suggero\t average MaxIoU: ', '{:.3}'.format(np.mean(suggeroScores)))

# ourScores, suggeroScores, fisherScores = [], [], []
# ourLens, suggeroLens, fisherLens   = [], [], []

# for k in res2.keys() : 
#     if 'Ours' in res2[k] : 
#         ourScores.extend(res2[k]['Ours']['scores'])
#         ourLens.extend(res2[k]['Ours']['lens'])
# #     if 'Fisher et. al.' in res2[k] : 
# #         fisherScores.extend(res2[k]['Fisher et. al.']['scores'])
# #         fisherLens.extend(res2[k]['Fisher et. al.']['lens'])
# #     if 'Suggero' in res2[k] : 
# #         suggeroScores.extend(res2[k]['Suggero']['scores'])
# #         suggeroLens.extend(res2[k]['Suggero']['lens'])

# print('Our\t average MaxIoU: ', '{:.3}'.format(np.mean(ourScores)))
# # print('Fisher\t average MaxIoU: ', '{:.3}'.format(np.mean(fisherScores)))
# # print('Suggero\t average MaxIoU: ', '{:.3}'.format(np.mean(suggeroScores)))

In [ ]:
res1 = defaultdict(dict)
res2 = defaultdict(dict)

for k in tqdm(dataset[:200]) : 
    logResult(k, model.greedyTree, 'Ours', res1)
#     logResult(k, autogroup, 'Fisher et. al.', res1)
#     logResult(k, suggero, 'Suggero', res1)
    
for k in tqdm(dataset[200:400]) : 
    logResult(k, model.greedyTree, 'Ours', res2)
#     logResult(k, autogroup, 'Fisher et. al.', res2)
#     logResult(k, suggero, 'Suggero', res2)

In [ ]:
ourScores, suggeroScores, fisherScores = [], [], []
ourLens, suggeroLens, fisherLens   = [], [], []

for k in res1.keys() : 
    if 'Ours' in res1[k] : 
        ourScores.extend(res1[k]['Ours']['scores'])
        ourLens.extend(res1[k]['Ours']['lens'])
#     if 'Fisher et. al.' in res1[k] : 
#         fisherScores.extend(res1[k]['Fisher et. al.']['scores'])
#         fisherLens.extend(res1[k]['Fisher et. al.']['lens'])
#     if 'Suggero' in res1[k] : 
#         suggeroScores.extend(res1[k]['Suggero']['scores'])
#         suggeroLens.extend(res1[k]['Suggero']['lens'])

print('Our\t average MaxIoU: ', '{:.3}'.format(np.mean(ourScores)))
# print('Fisher\t average MaxIoU: ', '{:.3}'.format(np.mean(fisherScores)))
# print('Suggero\t average MaxIoU: ', '{:.3}'.format(np.mean(suggeroScores)))

ourScores, suggeroScores, fisherScores = [], [], []
ourLens, suggeroLens, fisherLens   = [], [], []

for k in res2.keys() : 
    if 'Ours' in res2[k] : 
        ourScores.extend(res2[k]['Ours']['scores'])
        ourLens.extend(res2[k]['Ours']['lens'])
#     if 'Fisher et. al.' in res2[k] : 
#         fisherScores.extend(res2[k]['Fisher et. al.']['scores'])
#         fisherLens.extend(res2[k]['Fisher et. al.']['lens'])
#     if 'Suggero' in res2[k] : 
#         suggeroScores.extend(res2[k]['Suggero']['scores'])
#         suggeroLens.extend(res2[k]['Suggero']['lens'])

print('Our\t average MaxIoU: ', '{:.3}'.format(np.mean(ourScores)))
# print('Fisher\t average MaxIoU: ', '{:.3}'.format(np.mean(fisherScores)))
# print('Suggero\t average MaxIoU: ', '{:.3}'.format(np.mean(suggeroScores)))

## Distribution of MaxIoU with number of paths in group

Again, our advantage seems to come from doing well with the smaller groups of which there is a large number in this dataset. That is why we are winning here.

In [ ]:
ourLens_ = np.log2(ourLens)
fisherLens_ = np.log2(fisherLens)
suggeroLens_ = np.log2(suggeroLens)

def printScoresInLenRange(lens, scores, methodName, lo, hi) : 
    scores_ = [s for s, l in zip(scores, lens) if lo <= l <= hi]
    print(f'{methodName}\t', ':', 
          '{:.4}'.format(np.mean(scores_)), '+/-', 
          '{:.3}'.format(np.std(scores_)), 
          f'(bucket size = {len(scores_)})')

lenIntervals = [1, 3, 5, 6]

for lo, hi in zip(lenIntervals[:-1], lenIntervals[1:]) : 
    print("__________________________________________________________")
    print('Average MaxIoU for groups with #paths in range', 1 << lo, '-', 1 << hi)
    printScoresInLenRange(ourLens_, ourScores, 'Our', lo, hi)
    printScoresInLenRange(suggeroLens_, suggeroScores, 'Suggero', lo, hi)
    printScoresInLenRange(fisherLens_, fisherScores, 'Fisher', lo, hi)

print("__________________________________________________________")

In [ ]:
# Visualize all groups with paths in ranges 6 - 7 along with the predictions

def findBestMatchPathSet (ps, t) : 
    tpss = [t.nodes[_]['pathSet'] for _ in t.nodes]
    ious = [iou(ps, ps_) for ps_ in tpss]
    return tpss[argmax(ious)], max(ious)

lo = 1
hi = 2

keys = list(results.keys())
for T in keys[:10] :
    root = findRoot(T)
    pathsets = [T.nodes[n]['pathSet'] for n in T.nodes if n != root]
    for ps in pathsets : 
        if (1 << lo) <= len(ps) <= (1 << hi) : 
            tour = results[T]['Ours']['tree']
            tfisher = results[T]['Fisher et. al.']['tree']
            tsug = results[T]['Suggero']['tree']
            ops, oiou = findBestMatchPathSet(ps, tour)
            fps, fiou = findBestMatchPathSet(ps, tfisher)
            sps, siou = findBestMatchPathSet(ps, tsug)
            if oiou < fiou or oiou < siou : 
                print("___________________________________________________")
                print(T.svgFile)
                print(ps)
                plt.imshow(rasterize(T.doc, 200, 200))
                plt.show()
                plt.imshow(rasterize(subsetSvg(T.doc, ps), 200, 200))
                plt.show()
                print('Ours')
                print('MaxIoU:', oiou)
                plt.imshow(rasterize(subsetSvg(T.doc, ops), 200, 200))
                plt.show()
                treeImageFromGraph(tour)
                plt.show()
                print('Fisher')
                print('MaxIoU:', fiou)
                plt.imshow(rasterize(subsetSvg(T.doc, fps), 200, 200))
                plt.show()
                treeImageFromGraph(tfisher)
                plt.show()
                print('Suggero')
                print('MaxIoU:', siou)
                plt.imshow(rasterize(subsetSvg(T.doc, sps), 200, 200))
                plt.show()
                treeImageFromGraph(tsug)
                plt.show()


###### 

In [ ]:
svgFile = '../data/PublicDomainVectors/10723-Pale-sun-vector-image/10723-Pale-sun-vector-image.svg'
tree = SVGData(svgFile)
inferred = model.greedyTree(tree)
treeImageFromGraph(inferred)

In [ ]:
# Visualize the nearest neighbors for each node in the tree
nodeFeatures = dict()
embeddings = dict()

for n in inferred.nodes : 
    ps = tuple(inferred.nodes[n]['pathSet'])
    nodeFeatures[ps] = ThreeBranch.nodeFeatures(tree, ps, opts)
    tensorApply(
        nodeFeatures[ps],
        lambda x : x.to(opts.device).unsqueeze(0)
    )
    embeddings[ps] = model.embedding(nodeFeatures[ps])

for n in inferred.nodes : 
    ps = tuple(inferred.nodes[n]['pathSet'])
    similarity = dict()
    for m in inferred.nodes : 
        if n != m : 
            ps_ = tuple(inferred.nodes[m]['pathSet'])
            similarity[ps_] = negativeCosineSimilarity(embeddings[ps], embeddings[ps_]).item()
    top4 = sorted(list(similarity.keys()), key=lambda k: similarity[k])[:7]
    fig, axes = plt.subplots(1, 8, dpi=200)
    pss = [ps, *top4]
    crops = [deepcopy(nodeFeatures[_]['crop']) for _ in pss]
    ims = [im.squeeze().detach().cpu().numpy().transpose((1, 2, 0)) for im in crops]
    ims = [(im - im.min()) / (im.max() - im.min()) for im in ims]
    print(pss)
    print(['{:2f}'.format(-similarity[_]) for _ in pss[1:]])
    for im, ax in zip(ims, axes) : 
        ax.imshow(im)
        ax.set_xticks([], minor=True)
        ax.set_yticks([], minor=True)
        

    plt.show()
            